In [ ]:
import requests
import json
import datetime
import math

import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
base_url = "https://api.coronavirus.data.gov.uk/v1/data?"
filters = [
    "areaType=nation",
    "areaName=england",
]
structure = {
    "date": "date",
    "cases": "newCasesBySpecimenDate",
    "admissions": "newAdmissions",
    "deaths": "newDeaths28DaysByDeathDate",
}
params = {
    "filters": str.join(";", filters),
    "structure": json.dumps(structure, separators=(",", ":"))
}

response = requests.get(base_url, params=params)
data = response.json()
df0 = pd.DataFrame.from_records(data["data"])
df0 = df0.rolling(window=7, on="date").mean()
df0["date"] = pd.to_datetime(df0["date"])
df0["source"] = "data"
mask = df0["date"] >= "2020-05-15"  # roughly when targetted testing ended
df0 = df0[mask]

df0 = df0.set_index("date")

In [ ]:
dfs = []
for variable in ["cases", "admissions", "deaths"]:
    mask1 = df0[variable].notna()
    base_date = df0.loc[mask1].index.max() - datetime.timedelta(days=28)
    mask2 = df0.index > base_date
    mask = mask1 & mask2

    x = [(date - base_date).days for date in df0.loc[mask].index]
    y = df0.loc[mask, variable]
    ks = np.polyfit(x, np.log(y), 1, w=np.sqrt(y))

    xx = np.array(range(28+28*4))
    yy = math.exp(ks[1]) * np.exp(ks[0]*xx)

    dfs.append(
        pd.DataFrame({
            "date": [base_date + datetime.timedelta(days=int(x)) for x in xx],
            f"{variable}": yy,
            "source": "projection",
        }).set_index("date")
    )
dfs.append(df0)
df = pd.concat(dfs, sort=False)
df["admission-case-ratio"] = df["admissions"].shift(periods=-11) / df["cases"]
df["death-case-ratio"] = df["deaths"].shift(periods=-35) / df["cases"]
df = df.reset_index()

In [ ]:
for variable in ["cases", "admissions", "deaths"]:
    log_y = False
    mask = df["source"]=="data"
    max_y = df.loc[mask, variable].max()*1.05
    min_y = 1 if log_y else 0
    range_y = [min_y, max_y]
    fig = px.line(
        data_frame=df,
        x="date",
        y=variable,
        color="source",
        log_y=log_y,
        range_y=range_y,
    )
    fig.show()

In [ ]:
for variable in ["admission", "death"]:
    log_y = True
    mask = df["source"]=="data"
    max_y = df.loc[mask, f"{variable}-case-ratio"].max()*1.05
    min_y = 0 if not log_y else (5e-3 if variable == "admission" else 1e-4)
    range_y = [min_y, max_y]
    px.line(
        data_frame=df,
        x="date",
        y=f"{variable}-case-ratio",
        log_y=log_y,
        range_y=range_y,
    ).show()